In [2]:
from sklearn.feature_extraction.text import HashingVectorizer # Occurence method 
from sklearn.feature_extraction.text import CountVectorizer # Count method
from sklearn.feature_extraction.text import TfidfVectorizer #Tfidf method
import nltk
import pandas as pd

In [3]:
#import lexicon token list 
lexicon = pd.read_excel('all_lexicon_tokens_unigram.xlsx')['token'].tolist()

In [4]:
#import Android app dataset
df = pd.read_csv("result_listtoken.csv")
df

,Unnamed: 0,Unnamed: 0.1,name,sub_category,maturity_rating,maturity_rating_reason,app_url,description,level,Cidx,Oidx,Tidx,Crating,Orating,Trating,C#diff,O#diff,T#diff
0,0,0,10.5,Card,Teen,Simulated Gambling,https://play.google.com/store/apps/details?id=...,(Introduction) Ten Point Half(10.5) is a very ...,3,"[201, 107, 106, 378, 83, 44, 26, 31, 1, 0]","[53, 45, 44, 83, 31, 30, 50, 26, 1, 0]","[76, 75, 77, 201, 83, 26, 44, 31, 1, 0]","[1, 1, 1, 3, 1, 1, 3, 1, 1, 3]","[2, 1, 1, 1, 1, 1, 0, 3, 1, 3]","[1, 1, 1, 1, 1, 3, 1, 1, 1, 3]",7,7,8
1,1,1,101,Card,Everyone,NaN,https://play.google.com/store/apps/details?id=...,"There are many variations of the basic game, a...",1,"[167, 378, 201, 107, 106, 26, 83, 44, 31, 1]","[83, 77, 76, 75, 53, 50, 44, 26, 31, 1]","[201, 50, 76, 75, 77, 83, 26, 44, 31, 1]","[1, 3, 1, 1, 1, 3, 1, 1, 1, 1]","[1, 1, 1, 1, 2, 0, 1, 3, 1, 1]","[1, 0, 1, 1, 1, 1, 3, 1, 1, 1]",2,2,1
2,2,2,2017,Adventure,Everyone,NaN,https://play.google.com/store/apps/details?id=...,Hello you in your game Bunny helped Collect co...,1,"[25, 149, 262, 54, 62, 60, 28, 48, 8, 2]","[51, 25, 26, 60, 29, 54, 28, 8, 48, 2]","[29, 54, 25, 149, 60, 62, 48, 28, 8, 2]","[3, 1, 1, 1, 1, 2, 1, 1, 1, 1]","[1, 3, 3, 2, 1, 1, 1, 1, 1, 1]","[1, 1, 3, 1, 2, 1, 1, 1, 1, 1]",2,3,2
3,3,3,2017,Sports,Everyone,NaN,https://play.google.com/store/apps/details?id=...,The 2017 mobile app is the most complete app f...,1,"[372, 371, 126, 242, 241, 471, 661, 479, 253, 3]","[126, 242, 241, 55, 80, 95, 69, 37, 38, 3]","[246, 253, 134, 197, 242, 241, 268, 69, 126, 3]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 3, 1, 1, 1]",0,0,1
4,4,4,2048,Puzzle,Everyone,NaN,https://play.google.com/store/apps/details?id=...,2048 game is a funnysimple and addictive numbe...,1,"[24, 14, 22, 15, 11, 6, 5, 17, 9, 4]","[19, 17, 18, 24, 22, 6, 15, 11, 12, 4]","[16, 17, 15, 14, 12, 11, 9, 6, 5, 4]","[1, 1, 1, 1, 1, 0, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 0, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 0, 1, 1]",0,0,0
5,5,5,2048,Puzzle,Everyone,NaN,https://play.google.com/store/apps/details?id=...,2048 Game Tips & Tricks 1. Take It Slow 2048 i...,1,"[24, 14, 22, 15, 11, 6, 17, 4, 9, 5]","[23, 17, 18, 19, 11, 15, 12, 6, 9, 5]","[18, 17, 15, 14, 12, 11, 4, 9, 6, 5]","[1, 1, 1, 1, 1, 0, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 0, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 0, 1]",0,0,0
6,6,6,2048,Puzzle,Unrated,Warning _�_ content has not yet been rated. Un...,https://play.google.com/store/apps/details?id=...,2048 is a simple but very attractive math puzz...,0,"[24, 14, 22, 15, 5, 4, 11, 17, 9, 6]","[24, 17, 23, 18, 19, 15, 12, 11, 9, 6]","[19, 18, 17, 15, 14, 12, 5, 11, 9, 6]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]",9,9,9
7,7,7,2048,Puzzle,Everyone,NaN,https://play.google.com/store/apps/details?id=...,2048 is a singleplayer sliding block puzzle game.,1,"[19921, 19941, 19946, 19949, 19965, 19966, 199...","[17968, 18024, 18044, 18071, 18121, 28956, 289...","[21482, 21483, 21484, 21563, 21607, 21624, 216...","[1, 1, 1, 1, 1, 1, 2, 1, 1, 1]","[3, 1, 1, 3, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 3, 1, 1]",1,2,1
8,8,8,2048,Puzzle,Everyone,NaN,https://play.google.com/store/apps/details?id=...,"2048 is a super addictive brain game , try to ...",1,"[229, 71, 91, 65, 262, 163, 112, 16, 62, 8]","[58, 19, 23, 72, 65, 71, 57, 16, 46, 8]","[48, 112, 65, 91, 90, 89, 84, 60, 62, 8]","[1, 1, 0, 1, 1, 1, 1, 1, 1, 1]","[2, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 0, 1, 1, 1, 2, 1, 1]",0,1,1
9,9,9,2048,Strategy,Everyone,NaN,https://play.google.com/store/apps/details?id=...,"2048 is played on a gray 44 grid, with numbere...",1,"[24, 14, 22, 15, 11, 6, 5, 4, 17, 9]","[14, 24, 18, 23, 19, 12, 15, 11, 6, 9]","[16, 20, 18, 19, 17, 12, 15, 14, 11, 9]","[1, 1, 1, 1, 1, 0, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 0, 1]","[1, 0, 1, 1, 1, 1, 1, 1, 1, 1]",0,0,0


## Create Document Term Matrix

In [ ]:
# instantiate the vectorizer and remove stop words 
# ignore terms that apear in more than 50% of the documents
# only keep terms that appear in at least 2 documents

In [3]:
vect1 = HashingVectorizer(stop_words='english',max_df=0.5,min_df=2)
matrix1 = vect.fit_transform(df['description'])
# Do not have get_feature_names() function!! so use the CountVectorizer()

TypeError: __init__() got an unexpected keyword argument 'max_df'

In [10]:
vect2 = CountVectorizer(stop_words='english',max_df=0.5,min_df=2)
matrix2 = vect2.fit_transform(df['description'])

In [9]:
vect3 = TfidfVectorizer(stop_words='english',max_df=0.5,min_df=2)
matrix3 = vect3.fit_transform(df['description'])

In [11]:
Cmatrix = matrix2
Tmatrix = matrix3

In [10]:
Tmatrix = matrix3

In [8]:
matrix2[matrix2!=0]=1

In [9]:
Omatrix = matrix2

In [6]:
#don't write the matrix into df
vect_df = pd.DataFrame(matrix2.toarray(),columns = vect2.get_feature_names())

In [ ]:
vect_df.to_csv('2Document_Term_Count_Matrix.txt')

In [24]:
vect_df[vect_df!=0]=1

In [25]:
vect_df.to_csv('2Document_Term_Occurence_Matrix.csv')

In [6]:
vect3_df = pd.DataFrame(matrix3.toarray(),columns = vect3.get_feature_names())
vect3_df.to_csv('2Document_Term_Tfidf_Matrix.csv')

## Calculate Similarity

In [24]:
df = pd.read_csv('2Android_have_idx0328.csv')

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
Csim = cosine_similarity(Cmatrix)

In [19]:
Osim = cosine_similarity(Omatrix)

In [11]:
Tsim = cosine_similarity(Tmatrix)

In [20]:
Csim.shape

(36927, 36927)

In [12]:
#find the top 10 value index for each row
import numpy as np
N = 10

In [15]:
Csorted_row_idx = np.argsort(Csim,axis=1)[:,Csim.shape[1]-N::]

In [20]:
Osorted_row_idx = np.argsort(Osim,axis=1)[:,Osim.shape[1]-N::]

In [13]:
Tsorted_row_idx = np.argsort(Tsim,axis=1)[:,Tsim.shape[1]-N::]

In [16]:
df['2Cidx'] = pd.Series(Csorted_row_idx.tolist())

In [21]:
df['2Oidx'] = pd.Series(Osorted_row_idx.tolist())

In [14]:
df['2Tidx'] = pd.Series(Tsorted_row_idx.tolist())

In [28]:
df.to_csv('2Android_have_idx0328.csv')

In [17]:
df['level'] = df['level'].fillna(0).astype('int')

In [18]:
#create a appindex:level dictionary
dic = df['level'].to_dict()

In [25]:
df.ix[0,'2Tidx']

'[22733, 3893, 3892, 11132, 3889, 378, 3874, 3901, 3890, 0]'

In [26]:
def mapping(row):
    
    ckey = row['2Cidx'][1:-1].replace(' ','').split(',')
    row['2Crating'] = [int(dic[int(key)]) for key in ckey]
    
    okey = row['2Oidx'][1:-1].replace(' ','').split(',')
    row['2Orating'] = [int(dic[int(key)]) for key in okey]
    
    tkey = row['2Tidx'][1:-1].replace(' ','').split(',')
    row['2Trating'] = [int(dic[int(key)]) for key in tkey]
    
    return row

In [27]:
df = df.apply(mapping,axis=1)

In [29]:
def counting(row):
    clist = [n for n in row['2Crating'] if n!=0]
    row['2C#diff'] = sum(clist != np.array(row['level']))
    olist = [n for n in row['2Orating'] if n!=0]
    row['2O#diff'] = sum(olist!=np.array(row['level']))
    tlist = [n for n in row['2Trating'] if n!=0]
    row['2T#diff'] = sum(tlist!=np.array(row['level']))
    
    return row

In [30]:
df = df.apply(counting,axis=1)

In [33]:
df.to_csv('result_alltoken.csv')

In [32]:
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1,Unnamed: 0.1.1,name,sub_category,maturity_rating,maturity_rating_reason,app_url,description,...,T#diff,2Cidx,2Oidx,2Tidx,2Crating,2Orating,2Trating,2C#diff,2O#diff,2T#diff
0,0,0,0,0,10.5,Card,Teen,Simulated Gambling,https://play.google.com/store/apps/details?id=...,(Introduction) Ten Point Half(10.5) is a very ...,...,8,"[3892, 25126, 11132, 9387, 32153, 18411, 378, ...","[33663, 17847, 17843, 15066, 7646, 3874, 11132...","[22733, 3893, 3892, 11132, 3889, 378, 3874, 39...","[3, 3, 1, 3, 3, 1, 3, 3, 4, 3]","[3, 3, 3, 1, 1, 3, 1, 3, 3, 3]","[3, 3, 3, 1, 3, 3, 3, 3, 4, 3]",3,3,2
1,1,1,1,1,101,Card,Everyone,NaN,https://play.google.com/store/apps/details?id=...,"There are many variations of the basic game, a...",...,1,"[15744, 21245, 8614, 25314, 28779, 8615, 19130...","[19293, 19481, 28200, 3193, 21243, 10916, 8610...","[8612, 21244, 8615, 8610, 21243, 8614, 21245, ...","[1, 3, 3, 1, 1, 1, 1, 1, 1, 1]","[1, 3, 3, 1, 1, 1, 1, 1, 1, 1]","[1, 3, 1, 1, 1, 3, 3, 1, 1, 1]",2,2,3
2,2,2,2,2,2017,Adventure,Everyone,NaN,https://play.google.com/store/apps/details?id=...,Hello you in your game Bunny helped Collect co...,...,2,"[3048, 15519, 23506, 35929, 21382, 13566, 3507...","[28177, 13502, 26773, 1599, 17110, 8528, 31748...","[1008, 26773, 5378, 6038, 5402, 28177, 5403, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 3, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0,1,0
3,3,3,3,3,2017,Sports,Everyone,NaN,https://play.google.com/store/apps/details?id=...,The 2017 mobile app is the most complete app f...,...,1,"[16566, 18864, 9311, 27696, 19422, 19550, 3308...","[23592, 27696, 19422, 18864, 9311, 19550, 3308...","[27979, 18864, 9311, 19550, 27696, 19422, 1487...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0,0,0
4,4,4,4,4,2048,Puzzle,Everyone,NaN,https://play.google.com/store/apps/details?id=...,2048 game is a funnysimple and addictive numbe...,...,0,"[294, 291, 23, 19, 311, 320, 17, 312, 318, 4]","[286, 327, 12, 291, 19, 23, 311, 320, 318, 4]","[314, 15, 23, 19, 17, 311, 312, 320, 318, 4]","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0,0,0
5,5,5,5,5,2048,Puzzle,Everyone,NaN,https://play.google.com/store/apps/details?id=...,2048 Game Tips & Tricks 1. Take It Slow 2048 i...,...,0,"[282, 330, 17, 276, 6, 9, 327, 312, 11, 5]","[4061, 9668, 18258, 34826, 32500, 33451, 5566,...","[276, 312, 6, 330, 14, 282, 345, 11, 327, 5]","[1, 1, 1, 1, 0, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 0, 1, 1, 1, 1, 1, 1, 1]",0,0,0
6,6,6,6,6,2048,Puzzle,Unrated,Warning _�_ content has not yet been rated. Un...,https://play.google.com/store/apps/details?id=...,2048 is a simple but very attractive math puzz...,...,9,"[11, 294, 312, 276, 327, 19, 9, 17, 334, 6]","[19, 311, 29122, 21417, 334, 335, 9, 329, 327, 6]","[294, 15, 337, 345, 282, 14, 311, 19, 334, 6]","[1, 0, 1, 1, 1, 1, 1, 1, 1, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]","[0, 1, 1, 1, 1, 1, 1, 1, 1, 0]",8,9,8
7,7,7,7,7,2048,Puzzle,Everyone,NaN,https://play.google.com/store/apps/details?id=...,2048 is a singleplayer sliding block puzzle game.,...,1,"[4040, 4039, 4050, 34817, 26463, 326, 31666, 4...","[29446, 18204, 34884, 34832, 34821, 28828, 406...","[342, 326, 305, 284, 311, 314, 23, 8, 15, 7]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 0, 1, 1, 1, 0, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0,0,0
8,8,8,8,8,2048,Puzzle,Everyone,NaN,https://play.google.com/store/apps/details?id=...,"2048 is a super addictive brain game , try to ...",...,1,"[4, 306, 324, 23, 312, 311, 15, 17, 13905, 8]","[291, 322, 17, 318, 4, 320, 34548, 297, 23, 8]","[19, 17, 13905, 312, 314, 284, 311, 23, 15, 8]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0,0,0
9,9,9,9,9,2048,Strategy,Everyone,NaN,https://play.google.com/store/apps/details?id=...,"2048 is played on a gray 44 grid, with numbere...",...,0,"[334, 276, 14, 345